# Add model 3, which is keeping the number of producers and also the number of  movies|

In [1]:
import json
import os
import sys
import argparse
from collections import Counter, defaultdict
from random import shuffle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import factorial
from scipy import stats
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap
from scipy.stats import ks_2samp
from operator import itemgetter

src_dir = os.path.abspath(os.path.join(os.pardir, os.pardir,'src'))
sys.path[0] = src_dir
from parser.support import ROLES, CREDITS
from parser.my_mongo_db_login import DB_LOGIN_INFO
import parser.support as support
import network.shift_graph_maker as sgm
import network.network_generator as net_gen

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Reading data

In [3]:
movie_producer_df = net_gen.open_movie_data()

Loaded IMDb movies producing_gender_percentage
Got all_movies


# Building synthetic networks


* fixed values
    * number of movies per year
    * number of producers per year

* variables
    * number of people per team - producer_num_list *
    * number of movies per producer - occurence_list *
    * size of gaps - gap_list


In [4]:
unlistyfied_producer_df = net_gen.unlistify(movie_producer_df, 'producers')

In [5]:
#number of movies per producer of the 90s
unlistyfied_producer_df['producer_id'] = unlistyfied_producer_df.producers.apply(lambda x: x[0])

In [6]:
movies_per_producer = unlistyfied_producer_df.groupby('producer_id').count()['_id']

# Model 3

fixed number of movies, fixed number of producers
given number of movies per producer
given size of gaps per producer/movie

## Model 3- 0

fixed number of movies, fixed number of producers

team size round down/up of the mean

In [7]:
movie_list = net_gen.bootstrap(movies_per_producer, len(movies_per_producer))

In [8]:
total_num_producers = len(list(set([i[0] for i in unlistyfied_producer_df.producers.tolist()])))

## Assign genders

In [9]:
gender_df = support.get_staff_df('producers')[['_id', 'female_count', 'first_movie', 'last_movie', 'gender']]

In [10]:
females = sgm.generate_gender_seeds(gender_df)

In [11]:
original_producer_list = [i[0] for sublist in movie_producer_df.producers.tolist() for i in sublist]
seeds = [i for i in females if i in original_producer_list]
len(seeds)

1064

In [12]:
gender_df = support.get_staff_df('producers')[['_id', 'female_count', 'first_movie', 'last_movie', 'gender']]

In [13]:
unlistyfied_producer_df['gender'] = unlistyfied_producer_df.apply(net_gen.assign_gender, args=(gender_df,), axis=1)

In [14]:
movie_per_producer_gender = {}
for g, g_df in unlistyfied_producer_df.groupby('gender'):
    movies_per_producer = g_df.groupby('producer_id').count()['_id'].tolist()
    movie_per_producer_gender[g] = movies_per_producer

In [15]:
movies = []
for p, group in unlistyfied_producer_df.groupby('producer_id'):
    group_sorted = group.sort_values('year')
    diff = group_sorted.year.diff().values
    if len(group_sorted) > 20:
        print(len(group_sorted))
        print(p)
#         print(group_sorted.year.tolist())
        diff_ratio = Counter(diff[~np.isnan(diff)])
        diffs, occur = zip(*diff_ratio.items())
        diffs = np.array(diffs)
        occur = np.array(occur)
        print(diffs)
        print(occur/sum(occur))
        movies.append(group_sorted.producers.tolist())

24
nm0004976
[0. 1. 2.]
[0.65217391 0.30434783 0.04347826]
43
nm0005544
[3. 0. 1.]
[0.02380952 0.83333333 0.14285714]
23
nm0083696
[1. 0.]
[0.27272727 0.72727273]
22
nm0181202
[0. 1. 2. 3.]
[0.71428571 0.19047619 0.04761905 0.04761905]
24
nm0276059
[0. 1. 2.]
[0.65217391 0.30434783 0.04347826]
34
nm0586969
[0. 1.]
[0.75757576 0.24242424]
21
nm0732708
[0. 1. 2.]
[0.65 0.3  0.05]
21
nm0797451
[0. 2. 1.]
[0.6  0.05 0.35]
43
nm0918424
[3. 0. 1.]
[0.02380952 0.83333333 0.14285714]


In [16]:
gap_dict = net_gen.get_gaps(unlistyfied_producer_df)

###  Generate movies

In [52]:
number_of_producers_per_year = {}
for year, df in unlistyfied_producer_df.groupby('year'):
    producers = list(set([i[0] for i in df.producers.tolist()]))
    number_of_producers_per_year[year] = len(producers)

In [53]:
#generate new producers every round
generated_producers = net_gen.generate_producers(total_num_producers)
generated_producers_dict = {}
generated_producers_dict['female'] = generated_producers[:len(seeds)]
generated_producers_dict['male'] = generated_producers[len(seeds):]
total_movie_frame = movie_producer_df[['_id', 'producers', 'year', 'producer_num']].copy(deep=True)

print('iterate years to produce team numbers')
for year, df in total_movie_frame.groupby('year'):
    num_producers = number_of_producers_per_year[year] #duplicate producers are already dropped
    mean_size = np.mean(df.producer_num.tolist())
    mean_ceil = np.ceil(mean_size)
    mean_floor = np.floor(mean_size)
    #fix the team size to its mean
    df['producer_num'] = df.producer_num.apply(net_gen.team_size, args=(mean_ceil, mean_floor))
    total_movie_frame['producer_num'].update(df.producer_num)
total_movie_frame['producers'] = [[]]*len(total_movie_frame)
print('generate movies')
total_num_teams = total_movie_frame.producer_num.sum()

iterate years to produce team numbers


/home/staff/junelee/miniconda3/envs/movie_network/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/staff/junelee/miniconda3/envs/movie_network/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/staff/junelee/miniconda3/envs/movie_network/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy o

generate movies


/home/staff/junelee/miniconda3/envs/movie_network/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/staff/junelee/miniconda3/envs/movie_network/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/staff/junelee/miniconda3/envs/movie_network/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [54]:
# generate movies
movie_dict = net_gen.generate_movie_num(total_num_teams, generated_producers_dict, movie_per_producer_gender)

In [55]:
# assign gaps by gender
movie_dict_per_producer, gap_dict_per_producer = net_gen.assign_gaps(movie_dict, gap_dict)

In [56]:
# distribute movies
df = total_movie_frame.copy()

def distribute_movies(df, gap_dict_per_producer):
    """
    Distribute movies according to the gap dist
    Input:
        df - dataframe for movies
        gap_dict_per_producer - dictionary of gaps per producer
    Output:
        df - dataframe for movies now filled with producers
    """

    df = df.copy(deep=True)
    sorted_gap_dict_per_producer = sorted(gap_dict_per_producer.items(), key=lambda kv: (sum(kv[1]), len(kv[1])), reverse=True)
    producers = [p[0] for p in sorted_gap_dict_per_producer]

    for p in producers:
        gaps = gap_dict_per_producer[p]
        shuffle(gaps)
        first_movie = find_first_available_movie(df, sum(gaps))
        available_movies = find_unfilled_movies(df)
        # when is the producer's first active year
        start_year = df[df._id == first_movie].year.values[0]
        # find the years that the producer made movie
        working_years = calculate_years(start_year, gaps)
        chosen_movies = choose_movies(available_movies, working_years)
        df = add_producers(total_movie_frame, p, chosen_movies)
    return df

In [57]:
distribute_movies(total_movie_frame, gap_dict_per_producer)

,_id,producers,year,producer_num,availability
22884,tt0099622,"[KQ140246, KS203030, TE001305, NG763322, MJ256...",1990,5,5
22502,tt0100822,"[HU880567, UI292628, QM097594, YY925545, BC616...",1990,5,5
10973,tt0100935,"[JG334789, VA295986, CQ301270, PI626682, FC316...",1990,5,5
22559,tt0099819,"[NU102764, AG010843, RJ987526, ES292180]",1990,4,4
24509,tt0099796,"[MC466386, VL541920, CC946195, QS962888]",1990,4,4
2827,tt0099371,"[SL373992, QU234675, AW475664, GQ104393]",1990,4,4
10867,tt0099044,"[DZ028310, VK145415, HI848490, HG458739, MC867...",1990,5,5
24478,tt0100606,"[RD657770, GH415602, VQ744387, DI138572]",1990,4,4
2923,tt0098987,"[IE238740, EL216405, GP995599, EV797795, CB956...",1990,5,5
24465,tt0100036,"[MX710192, ZL513070, VN559238, HD563425, HE868...",1990,5,5


In [58]:
total_movie_frame

,_id,producers,year,producer_num,availability
22884,tt0099622,"[KQ140246, KS203030, TE001305, NG763322, MJ256...",1990,5,5
22502,tt0100822,"[HU880567, UI292628, QM097594, YY925545, BC616...",1990,5,5
10973,tt0100935,"[JG334789, VA295986, CQ301270, PI626682, FC316...",1990,5,5
22559,tt0099819,"[NU102764, AG010843, RJ987526, ES292180]",1990,4,4
24509,tt0099796,"[MC466386, VL541920, CC946195, QS962888]",1990,4,4
2827,tt0099371,"[SL373992, QU234675, AW475664, GQ104393]",1990,4,4
10867,tt0099044,"[DZ028310, VK145415, HI848490, HG458739, MC867...",1990,5,5
24478,tt0100606,"[RD657770, GH415602, VQ744387, DI138572]",1990,4,4
2923,tt0098987,"[IE238740, EL216405, GP995599, EV797795, CB956...",1990,5,5
24465,tt0100036,"[MX710192, ZL513070, VN559238, HD563425, HE868...",1990,5,5


In [22]:
def add_producers(df, p, movie_list):
    """
    Append producer p to the producer list in df for the movie list
    Input:
        df - dataframe of the movies and producers
        p - producer
        movie_list - list of the movies that producer participated in
    Output:
        df - dataframe with the appended producer
    """
    mask = df['_id'].isin(movie_list)
    df_valid = df[mask]
    df.loc[mask, 'producers'] += [p]
    return df



In [40]:
def find_first_available_movie(df, N):
    """
    get the first movie to start
    """
    # find movies that are not filled with producers
    df['availability'] = df.producers.apply(lambda x: len(x))
    df_available = df[df.availability < df.producer_num]
    # find movies that has the possibe years considering the sum of gaps N
    last_year = df.iloc[-1].year
    possible_year = last_year - N
    df_available = df_available[df_available.year <= possible_year]
    available_movie = np.random.choice(df_available._id.tolist())
    return available_movie

In [24]:
def calculate_years(start_year, gaps):
    """
    producers active year
    """
    years = [start_year]
    for g in gaps:
        years.append(years[-1]+g)
    return years

In [25]:
def find_unfilled_movies(df):
    df['availability'] = df.producers.apply(lambda x: len(x))
    df_available = df[df.availability < df.producer_num]
    available_movies = df_available.groupby('year')['_id'].apply(list).to_dict()
    return available_movies

In [26]:
def choose_movies(available_movies, working_years):
    """
    available_movies - {year:[list of movies]}
    workign_years - years that prodcer produced movie
    """
    participating_movies = []
    for key, values in Counter(working_years).items():
        participating_movies.extend(np.random.choice(available_movies[key], values, replace=False))
    return participating_movies

In [27]:
def append_producer(row, p, movies):
    if row._id in movies:
        row.producers.append(p)
    return row

### Make dataframe with the exact team sizes

In [ ]:
data_dir = '/home/projects/movie-network/data/synthetic_data/model_3_0/'
gender_dir = '/home/projects/movie-network/data/synthetic_data/genders/'

In [ ]:
num_schedules = 1
version_list = net_gen.make_version(num_schedules)

In [ ]:
for key,value in sorted(gap_dict_per_producer.items(),key=lambda i:(sum(i[1]), len(i[1])),reverse=True):
    print(key, value, sum(value))

In [ ]:
total_movie_frame.head()

In [ ]:
shift_dist = []
# for v in range(num_schedules):


#generate new producers every round
generated_producers = net_gen.generate_producers(total_num_producers)
generated_producers_dict = {}
generated_producers_dict['female'] = generated_producers[:len(seeds)]
generated_producers_dict['male'] = generated_producers[len(seeds):]
total_movie_frame = movie_producer_df[['_id', 'producers', 'year', 'producer_num']].copy(deep=True)
print('iterate years')
for year, df in total_movie_frame.groupby('year'):
    num_producers = number_of_producers_per_year[year] #duplicate producers are already dropped
    mean_size = np.mean(df.producer_num.tolist())
    mean_ceil = np.ceil(mean_size)
    mean_floor = np.floor(mean_size)
    #fix the team size to its mean
    df['producer_num'] = df.producer_num.apply(team_size, args=(mean_ceil, mean_floor))
    total_movie_frame['producer_num'].update(df.producer_num)
total_movie_frame['producers'] = np.nan
print('generate movies')
total_num_teams = total_movie_frame.producer_num.sum()
dict_movies = generate_movie_num(total_num_teams, generated_producers_dict, movie_per_producer_gender)
shift_dist.append(list(dict_movies.values()))
print('distribute movies')


    
for i, row in total_movie_frame.iterrows():
    producers, occurence = zip(*dict_movies.items())
    producers = np.array(producers)
    occurence = np.array(occurence)
    size = row.producer_num
    team = np.random.choice(producers, size, replace=False, p=occurence/sum(occurence))
    #assign list to cell
    total_movie_frame['producers'] = total_movie_frame['producers'].astype(object)
    total_movie_frame.at[i, 'producers'] = team
    for p in team:
        dict_movies[p] -= 1
        if dict_movies[p] == 0:
            del dict_movies[p]
            
            
            
#     total_movie_frame.to_json(os.path.join(data_dir, 'movies_3_0_{}.json'.format(version_list[v])), orient='split')
#     #save gender
#     generated_gender_df = pd.DataFrame(columns=['producer_id', 'gender'])
#     for g, producers in generated_producers_dict.items():
#         genders = [g for ii in producers]
#         appending_df = pd.DataFrame({'producer_id': producers, 'gender': genders})
#         generated_gender_df = generated_gender_df.append(appending_df)
#     generated_gender_df.to_json(os.path.join(gender_dir, 'movies_3_0', 'version_{}.json'.format(version_list[v])), orient='split')

In [ ]:
p_list = list(set([i for sublist in total_movie_frame.producers.tolist() for i in sublist]))

In [ ]:
for l in shift_dist:
    print(ks_2samp(l, movie_per_producer_gender['male']+movie_per_producer_gender['female']))

# Model 2-1

fixed number of movies, fixed number of producers

team size sampled from the real schedule

## Fill in producers

In [ ]:
num_schedules = 10

version_list = []
while len(version_list) < num_schedules:
    ver = make_version()
    if ver not in version_list:
        version_list.append(ver)

In [ ]:
from os import listdir
from os.path import isfile, join

original_file_list = [join(original_dir, f) for f in listdir(original_dir) 
                       if isfile(join(original_dir, f))]

## Concat over multiple years

In [ ]:
movie_producer_df[['_id', 'producers', 'year', 'producer_num']].copy(deep=True).head()

In [ ]:
data_dir = '/home/projects/movie-network/data/synthetic_data/model_2_1/'

shift_dist = []
for v in range(num_schedules):
    #generate new producers every round
    generated_producers = net_gen.generate_producers(total_num_producers)
    generated_producers_dict = {}
    generated_producers_dict['female'] = generated_producers[:len(seeds)]
    generated_producers_dict['male'] = generated_producers[len(seeds):]
    total_movie_frame = movie_producer_df[['_id', 'producers', 'year', 'producer_num']].copy(deep=True)
    total_movie_frame['producers'] = np.nan
    print('generate movies')
    total_num_teams = total_movie_frame.producer_num.sum()
    dict_movies = generate_movie_num(total_num_teams, generated_producers_dict, movie_per_producer_gender)
    shift_dist.append(list(dict_movies.values()))
    print('distribute movies')
    for i, row in total_movie_frame.iterrows():
        producers, occurence = zip(*dict_movies.items())
        producers = np.array(producers)
        occurence = np.array(occurence)
        size = row.producer_num
        team = np.random.choice(producers, size, replace=False, p=occurence/sum(occurence))
        total_movie_frame['producers'] = total_movie_frame['producers'].astype(object)
        total_movie_frame.at[i, 'producers'] = team
        for p in team:
            dict_movies[p] -= 1
            if dict_movies[p] == 0:
                del dict_movies[p]
    total_movie_frame.to_json(os.path.join(data_dir, 'movies_2_1_{}.json'.format(version_list[v])), orient='split')
    #save gender
    generated_gender_df = pd.DataFrame(columns=['producer_id', 'gender'])
    for g, producers in generated_producers_dict.items():
        genders = [g for ii in producers]
        appending_df = pd.DataFrame({'producer_id': producers, 'gender': genders})
        generated_gender_df = generated_gender_df.append(appending_df)
    generated_gender_df.to_json(os.path.join(gender_dir, 'movies_2_1', 'version_{}.json'.format(version_list[v])), orient='split')

In [ ]:
for l in shift_dist:
    print(ks_2samp(l, movie_per_producer_gender['male']+movie_per_producer_gender['female']))

# Problem

Random sampling does not account for all of the producers

In [ ]:
ver

In [ ]:
from os.path import normpath, basename, join, isfile
from os import listdir

In [ ]:
network_model =  basename(normpath(data_dir))
gender_folder = os.path.join('/home/projects/movie-network/data/synthetic_data/genders/', network_model)

In [ ]:
gender_file = [join(gender_folder, f) for f in listdir(gender_folder) if isfile(join(gender_folder, f)) and ver in f]

In [ ]:
if len(gender_file) == 1:
    gender_file = gender_file[0]
else:
    raise IndexError('the version has duplicate or it does not exist')

In [ ]:
df_gender = pd.read_json(gender_file, orient='split')
seeds = df_gender[df_gender.gender=='female'].producer_id.tolist()

In [ ]:
len(seeds)

In [ ]:
from os.path import dirname, realpath
two_up = dirname(dirname(__file__))

In [ ]:
os.path.abspath(os.path.join(data_dir, os.pardir, 'gender'))

In [ ]:
data_dir